### Get the Data

In [3]:
df <- read.csv('cancer.csv')

In [4]:
head(df)

mean.radius,mean.texture,mean.perimeter,mean.area,mean.smoothness,mean.compactness,mean.concavity,mean.concave.points,mean.symmetry,mean.fractal.dimension,...,worst.texture,worst.perimeter,worst.area,worst.smoothness,worst.compactness,worst.concavity,worst.concave.points,worst.symmetry,worst.fractal.dimension,target
17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0


In [6]:
colnames(df)

[1] "mean.radius"             "mean.texture"           
 [3] "mean.perimeter"          "mean.area"              
 [5] "mean.smoothness"         "mean.compactness"       
 [7] "mean.concavity"          "mean.concave.points"    
 [9] "mean.symmetry"           "mean.fractal.dimension" 
[11] "radius.error"            "texture.error"          
[13] "perimeter.error"         "area.error"             
[15] "smoothness.error"        "compactness.error"      
[17] "concavity.error"         "concave.points.error"   
[19] "symmetry.error"          "fractal.dimension.error"
[21] "worst.radius"            "worst.texture"          
[23] "worst.perimeter"         "worst.area"             
[25] "worst.smoothness"        "worst.compactness"      
[27] "worst.concavity"         "worst.concave.points"   
[29] "worst.symmetry"          "worst.fractal.dimension"
[31] "target"

### Building the Model
We'll need the e1071 library.

In [7]:
#install.packages('e1071',repos = 'http://cran.us.r-project.org')

In [8]:
library(e1071)

In [9]:
help(svm)

#### Train Test split 

In [36]:
df$target <- factor(df$target)

In [37]:
library(caTools)
set.seed(101)

split = sample.split(df$target, SplitRatio = 0.70)

final.train = subset(df, split == TRUE)
final.test = subset(df, split == FALSE)

### Model on Train data

In [38]:
model <- svm(target ~ ., data=final.train)

In [39]:
summary(model)


Call:
svm(formula = target ~ ., data = final.train)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.03333333 

Number of Support Vectors:  98

 ( 51 47 )


Number of Classes:  2 

Levels: 
 0 1




### Example Predictions

In [42]:
predicted.values.test <- predict(model,final.test)

In [43]:
table(predicted.values.test,final.test$target)

                     
predicted.values.test   0   1
                    0  63   1
                    1   1 106

### Advanced - Tuning

In [50]:
x.train <- subset(final.train, select = -c(target))
y.train <- subset(final.train, select = c(target))

In [53]:
  obj <- tune(svm, target~., data = final.train, 
              ranges = list(gamma = 2^(-1:1), cost = 2^(2:4)),
              tunecontrol = tune.control(sampling = "fix")
             )

In [55]:
summary(obj)
  


Parameter tuning of 'svm':

- sampling method: fixed training/validation set 

- best parameters:
 gamma cost
   0.5    4

- best performance: 0.2556391 

- Detailed performance results:
  gamma cost     error dispersion
1   0.5    4 0.2556391         NA
2   1.0    4 0.3533835         NA
3   2.0    4 0.3609023         NA
4   0.5    8 0.2556391         NA
5   1.0    8 0.3533835         NA
6   2.0    8 0.3609023         NA
7   0.5   16 0.2556391         NA
8   1.0   16 0.3533835         NA
9   2.0   16 0.3609023         NA


In [56]:
help(tune)

In [58]:
obj$best.model


Call:
best.tune(method = svm, train.x = target ~ ., data = final.train, 
    ranges = list(gamma = 2^(-1:1), cost = 2^(2:4)), tunecontrol = tune.control(sampling = "fix"))


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  4 
      gamma:  0.5 

Number of Support Vectors:  374


In [59]:
predicted.values.test <- predict(obj$best.model,final.test)

In [60]:
table(predicted.values.test,final.test$target)

                     
predicted.values.test   0   1
                    0  44   2
                    1  20 105